In [1]:
#import necessary dependecies
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
warnings.filterwarnings('ignore')
from typing import List

In [2]:
#Data Path
data_path ='/Users/DELL/Desktop/Hackhathons/Zimnat_recommend/'
input_path ='/Users/DELL/Desktop/Hackhathons/Zimnat_recommend/'
save_path = '/Users/DELL/Desktop/Hackhathons/Zimnat_recommend/'
trains='Train.csv'
tests='Test.csv'
submissions='SampleSubmission.csv'

In [3]:
#define some fxn to read the data set
def read_train():
    train =pd.read_csv(data_path + trains, parse_dates=['join_date'])
    return train

def read_test():
    test =pd.read_csv(data_path + tests, parse_dates=['join_date'])
    return test

def read_submission():
    submission =pd.read_csv(data_path + submissions,)
    return submission

#Assign defined fxns to variables
train = read_train();test=read_test();submission=read_submission()

In [4]:
#print all dataset shape
print(train.shape, test.shape,submission.shape)

(29132, 29) (10000, 29) (210000, 2)


In [5]:
#Store Products ID in Multidimensional array
products =train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t =test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

In [6]:
#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

In [7]:
#A merger
train['combiner']='x'

In [8]:
#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

In [9]:
#preview random 3 sample
data.sample(3)

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,combiner
553730,G78TEUG,2018-01-09,F,M,1973,748L,2A7I,T4MS,GHYX,0,x
580966,U76A15J,2019-01-04,M,S,1996,94KC,2A7I,T4MS,GHYX,0,x
415876,19CA8OW,2012-01-05,M,M,1959,XX25,6E4H,90QI,K6QO,0,x


In [10]:
#Confirm Distribution (TEST AND SAMPLE SUBMISSION FILE)
print('==' * 18);print('TRAIN LABEL DISTRIBUTION');print('==' * 18);print(train['Label'].value_counts())
print('==' * 18);print('TEST LABEL DISTRIBUTION');print('==' * 18);print(test['Label'].value_counts())
print('==' * 18);print('SAMPLE SUBMISSION LABEL DISTRIBUTION');print('==' * 18);print(submission['Label'].value_counts())

TRAIN LABEL DISTRIBUTION
0    545419
1     66353
Name: Label, dtype: int64
TEST LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64
SAMPLE SUBMISSION LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64


In [11]:
train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)
print(train.shape, test.shape, submission.shape)

(611772, 10) (210000, 9) (210000, 2)


In [12]:
#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

In [13]:
#Re-arrange column (not necessary)
train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']]

In [14]:
#Lets Check for data co-existence in Train/Test
for col in ['PCODE','occupation_code', 'branch_code','occupation_category_code']:
    train_col = train[col].unique();test_col = test[col].unique()
    print ('Not in Test ' + col + ' ',[i  for i in train_col if i not in test_col])
    print ('Not in Train ' + col + ' ',[i  for i in test_col  if i not in train_col]);print('==' * 30)

Not in Test PCODE  []
Not in Train PCODE  []
Not in Test occupation_code  ['IE90', 'Q0LY', 'JSAX', '6XXU', '8HRZ', 'OQMY', 'INEJ', 'VZN9', 'UC7E', 'PSUY', 'WSRG', 'JQH3', 'LGTN', '738L', 'QQUP', '2XZ1', 'CAAV', 'LLLH', 'W1X2', 'DHSN', 'IX8T', '2US6', 'ZWPL', 'MEFQ', '9B5B', 'JUIP', 'BFD1', 'A4ZC', 'IMHI', 'E5PF', 'GZA8', '3YQ1', 'PJR4', 'NDL9', 'PPNK', '2686', '5LNN', '374O', 'URYD', 'M0WG', 'KBWO', 'ONY7', 'VYSA', 'KUPK', 'R7GL', 'HSVE', 'BER4', '6SKY', 'RH2K', 'ZHC2', 'W3ZV', 'FLXH', 'UYDZ', 'YJXM', '59QM']
Not in Train occupation_code  ['0ZND', '8CHJ', '9F96', 'HSI5', '93OJ', 'BIA0', 'E2MJ', 'JBJP', '0FOI']
Not in Test branch_code  []
Not in Train branch_code  []
Not in Test occupation_category_code  []
Not in Train occupation_category_code  []


In [15]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS,P5DA,4WKQSBB X P5DA,0
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS,P5DA,CP5S02H X P5DA,0
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI,P5DA,2YKDILJ X P5DA,0
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI,P5DA,2S9E81J X P5DA,0
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS,P5DA,BHDYVFT X P5DA,0


In [16]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,F86J5PC,2018-01-12,M,M,1984,94KC,DZRV,90QI,P5DA,F86J5PC X P5DA
1,H6141K3,2019-01-10,M,M,1996,1X1H,J9SY,90QI,P5DA,H6141K3 X P5DA
2,RBAYUXZ,2020-01-01,F,W,1968,UAOD,2A7I,T4MS,P5DA,RBAYUXZ X P5DA
3,KCBILBQ,2019-01-02,M,M,1989,94KC,2A7I,T4MS,P5DA,KCBILBQ X P5DA
4,LSEC1ZJ,2020-01-02,F,M,1982,UAOD,0KID,T4MS,P5DA,LSEC1ZJ X P5DA


In [17]:
# LABEL ENCODE
def encode_LE(train,test,cols,verbose=True):
    for col in cols:
        df_comb = pd.concat([train[col],test[col]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        nm = col
        if df_comb.max()>32000: 
            train[nm] = df_comb[:len(train)].astype('int32')
            test[nm] = df_comb[len(train):].astype('int32')
        else:
            train[nm] = df_comb[:len(train)].astype('int16')
            test[nm] = df_comb[len(train):].astype('int16')
        del df_comb; x=gc.collect()
        if verbose: print(nm,', ',end='')

In [18]:
encode_LE(train, test, ['ID','branch_code', 'occupation_code','occupation_category_code','PCODE','sex','marital_status'])

ID , branch_code , occupation_code , occupation_category_code , PCODE , sex , marital_status , 

In [19]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,2019-01-02,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,2019-01-06,0,2,1981,11,19,5,15,CP5S02H X P5DA,0
2,3217,2013-01-06,1,6,1991,3,185,1,15,2YKDILJ X P5DA,0
3,3010,2019-01-08,1,2,1990,0,82,0,15,2S9E81J X P5DA,0
4,12546,2019-01-08,1,2,1990,3,157,5,15,BHDYVFT X P5DA,0


In [20]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,2018-01-12,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,2019-01-10,1,2,1996,0,128,1,15,H6141K3 X P5DA
2,29677,2020-01-01,0,7,1968,11,19,5,15,RBAYUXZ X P5DA
3,22120,2019-01-02,1,2,1989,4,19,5,15,KCBILBQ X P5DA
4,23700,2020-01-02,0,2,1982,11,3,5,15,LSEC1ZJ X P5DA


# TRAIN AND TEST FILE PREPARED

## READ THE INSTRUCTION CAREFULLY AND CRACK THE REST FOR PREDICTION